Import Libraries

In [3]:
import sqlite3
import requests
from tqdm import tqdm

from flask import Flask, request
import json 
import numpy as np
import pandas as pd

# Introduction
In most cases when we are running a business, there are a lot of data stakeholder outside our company. The problem is that we need to provide the access in a way that they will not break our security rules or concerns. One way to solve that is by creating an API for the database. In this project, we will introduce you on how python is used for data transaction management using Flask API. 

**Usecase**: Bikeshare App\
Have you ever rent a bike for faster mobility in town? In the past few years, this business once become a phenomenom. In Indonesia, there are lots of similar services, for example, the Jakarta government's "GOWES" bike sharing service that launcehd in July 2020. 

For the user perspective, the general journey is denoted as follows:
- User scan the bike located at some bike station, sending the data to database as the intent of "start riding"
- Once user has reached its destination station, he/she put back the bike, sending the data again to the database as the intent of "finished riding"

For each activity, there are data transactions between user and database. And how do you think each user's phone communicate with the server for storing and receiving the data? Using API ofcourse! 

We will later create a simplified version of the API service which handles data transactions and analysis. 


**Goals**: Make an API service to connect 3rd party user with data using HTTP request

**End Product**: A Flask API which capable of doing: 
- Input new data to database
- Read specific data from database
- Get specific insight from data analysis process (ie: best performing stations)

**Scoring Metrics**: 

1. 1 point - Created Flask App

-  create app.py file to make flask app
-  create Flask app to execute all of the endpoint you have made

2. 2 points - Created functionality to read or get specific data from the database

- create query to read data from database
- create function to execute read specific information into table from database

3. 4 points - Created functionality to input new data into each table for the databases

- create query to insert new data into stations and trips table
- create function to execute input data into stations and trips table

4. 3 points - Created static endpoints which return analytical result (must be different from point 2,3)

- create query to make analytical resultfrom the data
- create static endpoint to analyze the data from database, for example average trip durations
    
5. 3 points - Created dynamic endpoints which return analytical result (must be different from point 2,3,4)

- create query to make analytical resultfrom the data
- create dinamic endpoint to analyze the data from database, for example average trip durations for each bike_id

6. 3 points - Created POST endpoint which receive input data, then utilize it to get analytical result (must be different from point 2,3,4,5)

- create input data for refering into query for post endpoint
- make query and aggregation function to implement the input

**Tools**: 
- **Python** with **Jupyter Notebook**, installed with **required libraries**
- **Visual Studio Code (VSCode)**: Recommended for writing application scripts
- **TablePlus**: Recommended for easy database access and exploration
- Postman: Optional for easy API testing


# About the Data

The data that we will be using in this project is **[Austin Bike Share]()** dataset which contains information on bike trip start location, stop location, duration, type of bike share user in the city of Austin, Texas. Bike station location data is also provided. 

All the information is stored in a database called **austin_bikeshare.db**. However, we also provides the non existing data in csv files. These data then will be imported into the database using the API

Lists of files:
- **austin_bikeshare.db**: The database, contains `trips` and `stations` table
- **data/austin_bikeshare_stations.csv**: contains all the stations information which is not yet available in the database table
- **data/austin_bikeshare_trips_2021.csv**: contains all 2021 data which is not yet available in the database table

In [4]:
# Reading the csv data
trips = pd.read_csv('data/austin_bikeshare_trips_2021.csv')
stations = pd.read_csv('data/austin_bikeshare_stations.csv')

## Taking a look for trips data
`trips` table in database, or `austin_bikeshare_trips.csv` in original files. It roughly consisted of 1.3 million rows

In [3]:
trips.head(2)

,trip_id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,23455589,Local365,174,2021-01-26 17:47:42 UTC,4059.0,Nash Hernandez/East @ RBJ South,3660.0,East 6th/Medina,7
1,23459960,Local365,19265,2021-01-28 08:03:52 UTC,4054.0,Rosewood/Chicon,4055.0,11th/Salina,16


**Data Descriptions**:
- `trip_id`: unique trip id int
- `subscriber_type`: membership typ e.g. walk up, annual, other bike share, etc
- `bikeid`: integer id of bike
- `start_time`: YYYY-MM-DD HH:MM:SS
- `startstationid`: integer id of start station
- `startstationname`: string of start station name
- `endstationid`: integer id of end station
- `endstationname`: string of end station name
- `duration_minutes`: int minutes of trip duration

**[nanti kita yang ngisi]**
- `month`: month, integer
- `year`: year of trip, int
- `checkout_time`: HH:MM:SS, see start time for date stamp

## Taking a look for stations data
`stations` table in database, or `austin_bikeshare_stations.csv` in original files 

In [4]:
stations.tail(2)

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
96,3795,Dean Keeton & Whitis,active,300 W. Dean Keeton St.,NaN,32671.0,undetermined_parking,19.0,solar,55.0,5.0,NaN,9,2021-01-04T12:00:00Z
97,3797,21st & University,active,2007 University Ave.,NaN,32674.0,undetermined_parking,19.0,solar,55.0,5.0,ROW might belong to UT,9,2021-01-04T12:00:00Z


**stations table (or austin_bikeshare_trips.csv)**:
- `station_id`: integer id of station
- `name`: string of station name
- `status`: string of station status (active, closed, moved, ACL-only)
- `address`: string of station address
- `alternate_name`: string of station alternative name
- `city_asset_number`: integer of station's asset number
- `property_type`: string of station's property type
- `number_of_docks`: integer of number of available bike docks
- `power_type`: string of station's power source type
- `footprint_length`: float of station' blueprint length (size of the station). Probably in meters
- `footprint_width`: float of station' blueprint width (size of the station). Probably in meters
- `notes`: string of additional notes
- `council_district`: integer of stations's council district
- `modified_date`: date of last modified information regarding the station

# Database

The first important task is to make sure we can securely make connections to the database. In this scenario, we will connect to sqlite database, `austin_bikeshare.db` via python. In this part, we will re-visit on how to work with databases, started with making a connection, get some data, and insert data into it. 

You can directly connect and view the database using TablePlus, or run the following code to create the connection

In [5]:
# Define a function to create connection for reusability purpose
def make_connection():
    connection = sqlite3.connect('austin_bikeshare.db')
    return connection

conn = make_connection()
make_connection()

## POST: Insert data into database

Generally, POST method will utilize the data sent by user for specific purporse, for example: 

- Insert new data into the database. 
- Operate the data into some function

We will learn how to insert data into a specific table in our database. Please refer to the following code to create and run the query for the given task

**insert query perintahnya**
> INSERT INTO <TABLE> VALUES (VALUE1, VALUE2, ..., VALUEn)

**code breakdown**
1. stations.iloc[nomor] << ini buat ngambil satu baris (per transaksi i sppose)
> karena diatas dia maunya datanya diapit kurung biasa (xxx) tapi kalo iloc dia ngasilin data series
2. tambahin .values buat ekstrak datanya arraynya (dia masi ada []nya (i sppose))
3. pake tuple() buat ngeluarin data yang () aja
> query gabisa recognize NaN
4. jadi harus .fillna('')

In [21]:
# Get the data values
data = tuple(stations.iloc[10].fillna('').values)

# Make the query
query = f"""
INSERT INTO stations
VALUES {data}
"""

In [22]:
# See the actual query looks like
print(query)


INSERT INTO stations
VALUES (2712, 'Toomey Rd @ South Lamar', 'closed', '1301 Toomey Road', '', '', '', '', '', '', '', '', 5, '2021-01-04T12:00:00Z')



In [7]:
# Execute the query (gabisa execute data yang primary keynya sama) 
# ini bakal error semua soalnya datanya udah di up di database semua di bwh
# tapi udah bisa kok :D
conn.execute(query)

IntegrityError: UNIQUE constraint failed: stations.station_id

In [14]:
# Commit changes to database
conn.commit()

Once the code above succesful, it's recomended to wrap it into a function so that we can reuse it in the future. Complete the following codes to make the function:

In [8]:
def insert_into_stations(data, conn):
    query = f"""INSERT INTO stations values {data}"""
    try:
        conn.execute(query)
    except:
        return 'Error'
    conn.commit()
    return 'OK'

In [11]:
# Example use of the function 
conn = make_connection()
data = tuple(stations.iloc[7].fillna('').values) # Randomly select a data
result = insert_into_stations(data, conn)
result

'OK'

**TASK: Create a Function to insert data for `trips` table**

In [18]:
data_trips = tuple(trips.iloc[2].fillna('').values)

In [12]:
# Your code here
def insert_into_trips(data_trips, conn):
    query_trips = f'''INSERT INTO trips values {data_trips}'''
    try:
        conn.execute(query_trips)
    except:
        return 'Error'
    conn.commit()
    return 'OK'

In [13]:
conn = make_connection()
data_trips = tuple(trips.iloc[4].fillna('').values)
result_trips = insert_into_trips(data_trips, conn)
result_trips

'OK'

## GET: Read specific data

Generally, GET method will ask for specific the data in the database, alongside with additional information we might send. For example: 
- Get number of unique user
- Get full trips information for specific user id
- Get average trips duration and length for specific user id

In this example, we will implement the most basic GET method, that is to get Station information in the table based on specific Station ID

In [11]:
# Get Specific Station ID  Query
station_id = 2500
query_station_id = f"""
SELECT * FROM stations
WHERE station_id = {station_id}
"""

# Get all station ID Query
query_station_all = "SELECT * FROM stations"

In [12]:
print(query_station_id)


SELECT * FROM stations
WHERE station_id = 2500



In [13]:
# Run the query and get the result
result = pd.read_sql_query(query_station_id, conn)

In [14]:
result

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,2500,Republic Square,closed,425 W 4th Street,,,,,,,,,9,2021-01-04T12:00:00Z


Just like the previous section, create a function to read specific station

In [15]:
def get_station_id(station_id, conn):
    query_station_id = f"""SELECT * FROM stations WHERE station_id = {station_id}"""
    result = pd.read_sql_query(query_station_id, conn)
    return result 

def get_all_stations(conn):
    query_station_all = f"""SELECT * FROM stations"""
    result = pd.read_sql_query(query_station_all, conn)
    return result

**TASK: Create a Function to read data from `trips` table**

In [16]:
trip_id=23459960
query_trip_id = f"""
SELECT * FROM trips
WHERE trip_id = {trip_id}
"""

query_trip_all = f"SELECT * FROM trips"

In [23]:
# Your code here
def get_trip_id(trip_id, conn):
    query_trip_id = f"""SELECT * FROM trips WHERE id = {trip_id}"""
    result = pd.read_sql_query(query_trip_id, conn)
    return result
    
def get_all_trips(conn):
    query_trip_all = f"""SELECT * FROM trips"""
    result = pd.read_sql_query(query_trip_all, conn)
    return result

# Flask App

Flask is a micro-framework for python. Generally we can build any application out of it. To start with, let's make our first empy flask app. 
Creata a new file `app.py`, then copy-paste the following codes into it and save it. 

*Notes: it's recomended to use vscode as editor since it support vast linting supports, including python which we find really helpful in coding*

```python
from flask import Flask, request
app = Flask(__name__) 

if __name__ == '__main__':
    app.run(debug=True, port=5000)
```

To run the app, you can open your terminal, go to the specific folder and run `python app.py` using your designated virtual environment

## Routes and Endpoints

### Implement endpoints

If you succesfully run the previous app and access the localhost:5000 , you might get an 404 not found error. This is because we **have not** define yet what will happen if people accessing our root path of the app (localhost:5000/)

Add the following example of route or endpoint into your `app.py` just before the `if __name__ === '__main__':` line, and reload the page to see if it works

```python 
@app.route('/')
def home():
    return 'Hello World'
```

Above endpoints will runs the `home` function anytime user access the `/` page.

Since we are going to handle all the requests through this app, the very next step is to implement our functionalites. Let's start with implementing the functionality to read all station data

we will add the `get_all_stations()` functions into our app, create a `@app.route('/stations/')` endpoint and call the function in it. The code should looks like:

```python 
@app.route('/stations/')
def route_all_stations():
    conn = make_connection()
    stations = get_all_stations(conn)
    return stations.to_json()
    
def get_all_stations(conn):
    query = f"""SELECT * FROM stations"""
    result = pd.read_sql_query(query, conn)
    return result
```

However, adding just above codes is not enough. If you see, there is a dependency inside the function, which is `make_connection()` function. Other than that, we will be using all the required libraries. 

Hence, we can complete the code by : 
- import the required libraries at the top of the `app.py` file
- write the `make_connection()` function before any routes declarations

Once it's completed, you should see no errors in the vscode screen, and your endpoint should work just fine and returns all the stations. It's now your task to impelement the `get_all_trips` into the app 

**TASK: Implement `get_all_trips()` method into the app** \
Place it under `@app.route('/trips/')` endpoint

In [ ]:
# your code
@app.route('/trips/')
def route_all_trips():
    conn = make_connection()
    trips = get_all_trips(conn)
    return trips.to_json()

### Access Endpoints

To access our endpoints using python, simply use `requests` library followed by the request method (post, get, put, delete, etc) the passed in the url or data 

In [6]:
url = 'http://127.0.0.1:5000/stations/'
res = requests.get(url)

In [7]:
res

<Response [200]>

The response sent by the server was not only contains bare data. It was packed as HTTP response, meaning that we need to unpack the response first  in order to get the data. we can use `res.json()` to obtain it, then use pandas to transform it into dataframe for readibility or future analysis purposes. 

In [20]:
pd.DataFrame(res.json())#.head()

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,3464,Pease Park,closed,1155 Kingsbury St,,,,,,,,,9,2021-01-04T12:00:00Z
1,2500,Republic Square,closed,425 W 4th Street,,,,,,,,,9,2021-01-04T12:00:00Z
2,2536,Waller & 6th St.,closed,602 Waller St.,,,,,,,,,3,2021-01-04T12:00:00Z
3,2538,Bullock Museum @ Congress & MLK,closed,1881 Congress Ave.,,,,,,,,,1,2021-01-04T12:00:00Z
4,2541,State Capitol @ 14th & Colorado,closed,206 W. 14th St.,,,,,,,,,1,2021-01-04T12:00:00Z
5,2545,ACC - Rio Grande & 12th,closed,700 W. 12th St.,,,,,,,,,9,2021-01-04T12:00:00Z
6,2546,ACC - West & 12th Street,closed,1231 West Ave.,,,,,,,,,9,2021-01-04T12:00:00Z
7,2550,Republic Square @ Guadalupe & 4th St.,closed,Presented by Austin Ventures,,,,,,,,,9,2021-01-04T12:00:00Z


In [24]:
url = 'http://127.0.0.1:5000/trips/'
res = requests.get(url)
res

<Response [200]>

In [25]:
pd.DataFrame(res.json()).head(5)

,id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,9900289692,Walk Up,248,2015-10-02 21:12:01 UTC,1006,Zilker Park West,1008,Nueces @ 3rd,39
1,9900285987,24-Hour Kiosk (Austin B-cycle),446,2014-10-26 15:12:00 UTC,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
2,9900285989,24-Hour Kiosk (Austin B-cycle),203,2014-10-26 15:12:00 UTC,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
3,9900285991,24-Hour Kiosk (Austin B-cycle),101,2014-10-26 15:12:00 UTC,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,30
4,9900286140,24-Hour Kiosk (Austin B-cycle),242,2014-10-26 18:12:00 UTC,2541,State Capitol @ 14th & Colorado,2541,State Capitol @ 14th & Colorado,19


## Static and Dynamic Endpoints

On previous part we already made several endpoints which control how our server will react whenever users access it. 
If we take a look at the endpoints, it's all static : 
- `@app.route('/')`
- `@app.route('/home/')`
- `@app.route('/stations/')`
- `@app.route('/trips/')`


What if, instead of getting all the stations information, we only wanted to read a specific station information of station_id 3464 ? \
should we make an excact endpoint of that like `@app.route('stations/3464')`?. But what about the others? are we going to make a bunch of endpoints for each specific id like: 
- `@app.route('/stations/3464')`
- `@app.route('/stations/2500')`
- `@app.route('/stations/2541')`
- and so on...

Of course we shouldn't. One way to overcome the problem is by delcaring a **dynamic endpoints**. Basically, it's an endpoint which allows user to insert a variable values in it. The final looks of the dynamic endpoint version of above problem is: 
- `@app.route('stations/<station_id>')`

From above endpoint, the \<station_id\> is the variable, and we need to make sure that it influence how our following function acts. The whole dynamic endpoints for getting specific station by its id will looks like: 

```python 
@app.route('/stations/<station_id>')
def route_stations_id(station_id):
    conn = make_connection()
    station = get_station_id(station_id, conn)
    return station.to_json()
```


Before accesing the endpoint, make sure that `make_connection()` and `get_station_id()` functions are included inside `app.py` script so that it won't raise an error. Now for the final step, we can try to access the dynamic endpoints by changing the `station_id` in the following codes

In [26]:
station_id = 3464
url = f"http://127.0.0.1:5000/stations/{station_id}"

In [27]:
response = requests.get(url)
pd.DataFrame(response.json())

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,3464,Pease Park,closed,1155 Kingsbury St,,,,,,,,,9,2021-01-04T12:00:00Z


**TASK: Implement dynamic endpoints to read specific trip by its trip_id!** \
Place it under `@app.route('/trips/<trip_id>')` endpoint

In [30]:
trip_id = 23459960
url = f"http://127.0.0.1:5000/trips/{trip_id}"
response = requests.get(url)
pd.DataFrame(response.json())

,id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,23459960,Local365,19265,2021-01-28 08:03:52 UTC,4054,Rosewood/Chicon,4055,11th/Salina,16


## Handling JSON data as input

Sometimes, in order to make something happens in our API, we need the user to send us the data. In this case, we need to handle how we can get the data (which mostly sent as json format) and utilize it inside our endpoint functions. 

In order to achieve that, we will be using flask's `request` classes.


```python
@app.route('/json') 
def json_example():
    
    req = request.get_json(force=True) # Parse the incoming json data as Dictionary
    
    name = req['name']
    age = req['age']
    address = req['address']
    
    return (f'''Hello {name}, your age is {age}, and your address in {address}
            ''')
```

In [18]:
data = {
    "name" : "Andi", 
    "age" : 24,
    "address" : "Jakarta, Indonesia"
}

url = "http://127.0.0.1:5000/json"

In [19]:
res = requests.get(url, json=data)

In [20]:
res

<Response [200]>

In [21]:
res.text

'Hello Andi, your age is 24, and your address in Jakarta, Indonesia\n            '

Now that we already know how to handle json input, we can try to implement and endpoint in which we can insert new data into `stations` table. 

```python

@app.route('/stations/add', methods=['POST']) 
def route_add_station():
    # parse and transform incoming data into a tuple as we need 
    data = pd.Series(eval(request.get_json(force=True)))
    data = tuple(data.fillna('').values)
    
    conn = make_connection()
    result = insert_into_stations(data, conn)
    return result


```

In [26]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

In [9]:
data = stations.iloc[21].fillna('').to_dict()
data_json = json.dumps(data, cls=NpEncoder)
data_json

'{"station_id": 3687, "name": "Boardwalk West", "status": "active", "address": "300 E. Riverside Dr.", "alternate_name": "", "city_asset_number": 16683.0, "property_type": "parkland", "number_of_docks": 9.0, "power_type": "solar", "footprint_length": 30.0, "footprint_width": 5.0, "notes": "", "council_district": 9, "modified_date": "2021-01-04T12:00:00Z"}'

In [10]:
url = "http://127.0.0.1:5000/stations/add"
res = requests.post(url, json=data_json)

In [11]:
res

<Response [200]>

In [48]:
res.text

'OK'

In [49]:
# Sintaks for iteratively insert stations (from stations_csv) into stations table

for idx, data in tqdm(stations.iterrows()):
    data = data.fillna('').to_dict()
    data_json = json.dumps(data, cls=NpEncoder)
    
    url = "http://127.0.0.1:5000/stations/add"
    res = requests.post(url, json=data_json)

98it [00:00, 134.44it/s]


**TASK: Using the API, insert all the trips data (in csv) into the database by running the following codes**\
It might run differently on each devices, so you might get some rest while waiting for it to complete

In [50]:
for idx, data in tqdm(trips.iterrows()):
    data = data.fillna('').to_dict()
    data_json = json.dumps(data, cls=NpEncoder)
    
    url = "http://127.0.0.1:5000/trips/add"
    res = requests.post(url, json=data_json)

17982it [03:07, 95.95it/s] 


In [51]:
res

<Response [200]>

# Make your own analytic endpoints

Based on what we've learned before, create your own endpoints which returns analytical result such as contigency tables, aggregation tables, or even just a values.

*ps: if the return is dataframe/series, don't forget to change it into json with `.to_json()` method*


- [x] 1 point - Created Flask App
- [x] 4 points - Created functionality to input new data into each table for the databases
- [x] 2 poitns - Created functionality to read or get specific data from the database
- [ ] 3 points - Created static endpoints which return analytical result (must be different from point 2,3)
- [ ] 3 points - Created dynamic endpoints which return analytical result (must be different from point 2,3,4)
- [ ] 3 points - Created POST endpoint which receive input data, then utilize it to get analytical result (must be different from point 2,3,4,5)


## Create Static Endpoint(s)

You can use the following cell to try-out your function before implementing it as an endpoint.

After created the endpoint, make sure to implement it to `app.py` file.

In [ ]:
# Your Code Here 
@app.route('/trips/average_duration/') 
def route_average_stations():
    conn = make_connection()
    average_time_to_each_stations = get_average_duration(conn)
    return average_time_to_each_stations

In [17]:
def get_average_duration (conn):
    query_average_duration = f'''SELECT * FROM trips'''
    result = pd.read_sql_query(query_average_duration, conn)
    result_avg = pd.pivot_table(
                    data=result,
                    index='end_station_name',
                    values='duration_minutes',
                    aggfunc='mean'
                    )
    return result_avg.to_json()

get_average_duration(conn)

'{"duration_minutes":{"10th & Red River":64.1894150418,"10th\\/Red River":75.1933333333,"11th & Salina":46.9446721311,"11th & Salina ":71.8421733506,"11th & San Jacinto":40.7670979667,"11th\\/Congress @ The Texas Capitol":68.0450653447,"11th\\/Salina ":60.1012345679,"11th\\/San Jacinto":56.4087481146,"12th\\/San Jacinto @ State Capitol Visitors Garage":50.4222689076,"13th & San Antonio":34.8101659751,"13th\\/San Antonio":47.1952861953,"16th\\/San Antonio":35.8103448276,"17th & Guadalupe":28.473956044,"17th\\/Guadalupe":97.4742014742,"21st & Speedway @PCL":14.0697355273,"21st & University":16.8311748999,"21st\\/Guadalupe":25.9109892978,"21st\\/Speedway @ PCL":15.6476297126,"21st\\/University":22.8025066464,"22nd & Pearl":16.2504724906,"22nd\\/Pearl":31.7579439252,"23rd & Rio Grande":32.288529631,"23rd & San Jacinto @ DKR Stadium":17.5012231976,"23rd\\/Rio Grande":55.1081935327,"23rd\\/San Jacinto @ DKR Stadium":32.6774801962,"26th\\/Nueces":39.4704264099,"28th\\/Rio Grande":34.262568605

In [18]:
url = 'http://127.0.0.1:5000/trips/average_duration'
res = requests.get(url)
res

<Response [200]>

In [20]:
pd.DataFrame(res.json())

,duration_minutes
10th & Red River,64.189415
10th/Red River,75.193333
11th & Salina,46.944672
11th & Salina,71.842173
11th & San Jacinto,40.767098
...,...
Waller & 6th St.,26.285215
West & 6th St.,28.881014
Zilker Park,44.201111
Zilker Park West,29.306073


## Create Dynamic Endpoints 

You can use the following cell to try-out your function before implementing it as an endpoint.

After created the endpoint, make sure to implement it to `app.py` file.

In [ ]:
# Your Code Here
@app.route('/trips/average_duration/<bike_id>') 
def route_duration_perbike(bike_id):
    conn = make_connection()
    average_time_perbike = get_average_duration_perbike(bike_id, conn)
    return average_time_perbike

In [48]:
def get_average_duration_perbike(bike_id, conn):
    query_bike_id = f"""SELECT * FROM trips WHERE bikeid = {bike_id}"""
    result = pd.read_sql_query(query_bike_id, conn, parse_dates='start_time')
    result['days']=result['start_time'].dt.day_name()
    result_avg = pd.pivot_table(
                    data=result,
                    index='days',
                    values='duration_minutes',
                    aggfunc='mean'
                    )
    return result_avg.to_json()

,duration_minutes
days,
Friday,36.455056
Monday,20.621622
Saturday,37.954397
Sunday,28.905904
Thursday,38.307500
Tuesday,18.540094
Wednesday,15.264151


In [16]:
bike_id = 528
url = f"http://127.0.0.1:5000/trips/average_duration/{bike_id}"
response = requests.get(url)
response

<Response [200]>

In [51]:
pd.DataFrame(response.json())

,duration_minutes
Friday,27.248408
Monday,31.285714
Saturday,34.115987
Sunday,36.638554
Thursday,27.307692
Tuesday,41.003401
Wednesday,17.844068


## Create POST Endpoints

You can use the following cell to try-out your function before implementing it as an endpoint.

If you still find it quite difficult, here's an example case you might wanted to try on:
- input : a dictionary contained a datetime period
    - { "period" : "2015-08" }
- output:
    - Aggregation table of bike rent activities for each station in that specific period
- example code: 

```python 
input_data = request.get_json() # Get the input as dictionary
specified_date = input_data['period'] # Select specific items (period) from the dictionary (the value will be "2015-08")

# Subset the data with query 
conn = make_connection()
query = f"SELECT * FROM stations WHERE start_time LIKE ({specified_date}%)"
selected_data = pd.read_sql_query(query, conn)

# Make the aggregate
result = selected_data.groupby('start_station_id').agg({
    'bikeid' : 'count', 
    'duration_minutes' : 'mean'
})

# Return the result
return result.to_json()

```

After created the endpoint, make sure to implement it to app.py file.

In [ ]:
@app.route('/trips/period', methods=['POST'])
def route_activities_by_period():
    conn = make_connection()
    input_data = request.get_json(force=True)

    specific_period = input_data['period']
    average_activity_period = get_avg_act(specific_period, conn)

    return average_activity_period

In [8]:
period={'period':'2020-12'}

def get_activities_by_period(period, conn):
    
    specific_period = period['period']

    query_act = f'''SELECT * FROM trips WHERE start_time LIKE '{specific_period}%' '''
    selected_data = pd.read_sql_query(query_act, conn)
    result = selected_data.groupby('subscriber_type').agg({
    'bikeid' : 'count', 
    'start_station_name' : 'nunique', 
    'end_station_name' : 'nunique', 
    'duration_minutes' : 'mean'})
    
    return result

get_activities_by_period(period, conn)

,bikeid,start_station_name,end_station_name,duration_minutes
subscriber_type,,,,
24 Hour Walk Up Pass,739,63,67,61.426252
3-Day Weekender,296,61,60,33.942568
Annual Membership,12,7,9,52.666667
Explorer,567,63,65,41.834215
Local31,1790,73,72,28.606145
Local365,2990,74,74,27.015385
Local365+Guest Pass,14,6,7,11.500000
Pay-as-you-ride,451,66,66,56.964523
Single Trip (Pay-as-you-ride),617,59,61,65.844408


In [48]:
data_json = {"period" : "2020-12"}

url = "http://127.0.0.1:5000/trips/period"

res = requests.post(url, json=data_json)
res

<Response [200]>

In [49]:
pd.DataFrame(res.json())

,bikeid,start_station_name,end_station_name,duration_minutes
24 Hour Walk Up Pass,739,63,67,61.426252
3-Day Weekender,296,61,60,33.942568
Annual Membership,12,7,9,52.666667
Explorer,567,63,65,41.834215
Local31,1790,73,72,28.606145
Local365,2990,74,74,27.015385
Local365+Guest Pass,14,6,7,11.500000
Pay-as-you-ride,451,66,66,56.964523
Single Trip (Pay-as-you-ride),617,59,61,65.844408
U.T. Student Membership,264,39,38,16.920455


# Submission

After finishing your work of all the rubrics, the next step will be;

1. Prepare your `Bikeshare API.ipynb` file that has been edited with your code and wrangling data.
2. Prepare your `app.py` file for your Flask App. Make sure you have implemented all the endpoints to `app.py` (including your custom static, dynamic and post endpoints).
3. Submit your `Bikeshare API.ipynb` and `app.py` file in your github repository. The dataset is optional to post since it has big size to post. 

